In [ ]:
# default_exp data.format_qtmdata

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Format qtmdata
>  Format the Qualisys sports data preprocessed from Matlab 

In [ ]:
#export
import numpy as np
import pandas as pd
import argparse
import os

import sys
import inspect

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

from core.camera import project_point_radial
from core.runningpose_dataset import runningpose_cameras_extrinsic_params, runningpose_cameras_intrinsic_params

In [ ]:
#export
def parse_args():
    parser = argparse.ArgumentParser(
        description='Reformat qtmdata so that we can train.'
    )
    parser.add_argument(
        '--data-file', 
        dest='data_file',
        help='qtm text file that has been formated in matlab', 
        type=str
    )
    parser.add_argument(
        '--output-dir',
        dest='output_dir',
        help='directory for reformated keypoint data (default: ./)',
        default='./',
        type=str
    )
    parser.add_argument(
        '--cut-frame',
        dest='cut_frame',
        help='input what frame to cut the dataframe at.',
        type=int,
    )
    parser.add_argument(
        '--camera',
        dest='camera',
        help='which misqus camera (1, 2, 3) to use in runningpose dataset',
        type=int,
        choices=range(1, 4)
    )
    
    return parser.parse_args()

In [ ]:
#export
def convert_to_2D(data_3D, camera_num):
    """Returns the corresponding 2D list for a 3D dataframe"""
    # Get camera parameters.
    R = runningpose_cameras_extrinsic_params[camera_num]['rotation']
    T = np.array([runningpose_cameras_extrinsic_params[camera_num]['translation']]).T
    # Choose fx as the focal length since in theory fx and fy should be equal. 
    f = runningpose_cameras_intrinsic_params[camera_num]['focal_length'][0]
    c = np.array([runningpose_cameras_intrinsic_params[camera_num]['center']]).T
    k = runningpose_cameras_intrinsic_params[camera_num]['radial_distortion']
    p = runningpose_cameras_intrinsic_params[camera_num]['tangential_distortion']
    # Extract a keypoint column and calculate it to 2D.
    data_2D = []  
    for column in data_3D:
        col_data = data_3D[column].values
        x_data = col_data[0::3]
        y_data = col_data[1::3]
        z_data = col_data[2::3]
        data_world = np.array([x_data, y_data, z_data]).T
        Proj, _, _, _, _ = project_point_radial(data_world, R, T, f, c, k, p)
        data_2D.append(Proj)
    
    # TODO: Update so it returns a dataframe instead of a list. 
    # Get column names from input data_3D. 
    # TODO: Check the einsum in project_point_radial one more time. 
    return data_2D

In [ ]:
#hide
# Loads the textfiles
labels_np = np.loadtxt('updated_labels.txt', dtype = 'str')
data_3D = np.loadtxt('Tindra1.txt', dtype = 'float', delimiter= ',')

# Reformats the data to a dataframe
data_3D = pd.DataFrame(data_3D, index=labels_np).T

# Removes every other frame (3 rows corresponds to 1 frame.)
remove = True
for i in range(0, data_3D.shape[0]):
    if i % 3 == 0:
        remove = not(remove)

    if remove:
        data_3D = data_3D.drop(i)

data_3D = data_3D.reset_index(drop=True)

data_2D = convert_to_2D(data_3D, 0)

In [ ]:
#hide
R = runningpose_cameras_extrinsic_params[0]['rotation']
T = np.array([runningpose_cameras_extrinsic_params[0]['translation']]).T
k = runningpose_cameras_intrinsic_params[0]['radial_distortion']
c = np.array([runningpose_cameras_intrinsic_params[0]['center']]).T
p = runningpose_cameras_intrinsic_params[0]['tangential_distortion']
f = runningpose_cameras_intrinsic_params[0]['focal_length'][0]
data_world = None
for column in data_3D:
    col_data = data_3D[column].values
    x_data = col_data[0::3]
    y_data = col_data[1::3]
    z_data = col_data[2::3]
    data_world = np.array([x_data, y_data, z_data]).T


In [ ]:
#hide
print(data_world.shape)
print(c.shape)
N = data_world.shape[0]
print(N)
X = R.dot(data_world.T - T)
print(X.shape)
XX = X[:2,:] / X[2,:]
print(XX.shape)
r2 = XX[0,:]**2 + XX[1,:]**2
print(r2.shape)
print(k)
A = np.tile(k, (1, N))
print(A.shape)
print(3*343)
B = np.array([r2, r2**2, r2**3])
print(B.shape)
radial = 1 + np.einsum('ij,kl->il', np.tile(k,(1, N)), np.array([r2, r2**2, r2**3]))
print(radial.shape)
tan = p[0]*XX[1,:] + p[1]*XX[0,:]
print(tan.shape)
XXX = XX * np.tile(radial+tan,(2,1)) + np.outer(np.array([p[1], p[0]]).reshape(-1), r2)
print(XXX.shape)
print(f)

Proj = (f * XXX) + c
print(Proj.shape)

(343, 3)
(2, 1)
343
(3, 343)
(2, 343)
(343,)
[-0.045952, 0.137059, 0.0]
(1, 1029)
1029
(3, 343)
(1, 343)
(343,)
(2, 343)
107467.257813
(2, 343)


In [ ]:
#export
def main(args):
    """
    Loads the qtm data then removes unwanted keypoints.
    Then it infers new keypoints adds them. 
    Further we scale the dataset using a the norm vector between the 
    root and 'SpineThoracic2'. 
    """
    # Loads the textfiles
    labels_np = np.loadtxt('qtm_labels.txt', dtype = 'str')
    data_3D = np.loadtxt(args.data_file, dtype = 'float', delimiter= ',')

    # Reformats the data to a dataframe
    data_3D = pd.DataFrame(data_3D, index=labels_np).T

    # Remove unwanted keypoints
    data_3D = data_3D.drop(
        columns=[
            'HeadL', 'HeadR', 'Chest', 'LThighFrontLow', 'RThighFrontLow', 
            'LShinFrontHigh', 'RShinFrontHigh', 'LForefoot5', 'RForefoot5', 
            'LHeelBack', 'RHeelBack', 'LArm', 'RArm','WaistLFront', 'WaistL', 
            'WaistRFront', 'WaistR', 'LHand2', 'RHand2'
        ]
    )
    # Create "new" keypoints by finding the mean between specific keypoints
    left_elbow_3D = data_3D.loc[:, ['LElbowOut','LElbowIn']].mean(axis=1)
    right_elbow_3D = data_3D.loc[:, ['RElbowOut','RElbowIn']].mean(axis=1)

    left_wrist_3D = data_3D.loc[:, ['LWristIn','LWristOut']].mean(axis=1)
    right_wrist_3D = data_3D.loc[:, ['RWristOut','RWristIn']].mean(axis=1)

    left_knee_3D = data_3D.loc[:, ['LKneeOut','LKneeIn']].mean(axis=1)
    right_knee_3D = data_3D.loc[:, ['RKneeOut','RKneeIn']].mean(axis=1)

    left_ankle_3D = data_3D.loc[:, ['LAnkleOut','LAnkleIn']].mean(axis=1)
    right_ankle_3D = data_3D.loc[:, ['RAnkleOut','RAnkleIn']].mean(axis=1)

    # Remove the keypoints that was taken as a mean
    data_3D = data_3D.drop(
        columns=[
            'LElbowOut','LElbowIn', 'RElbowOut','RElbowIn', 
            'LWristIn','LWristOut', 'RWristIn','RWristOut', 
            'LKneeIn', 'LKneeOut','RKneeIn', 'RKneeOut',
            'LAnkleOut','LAnkleIn','RAnkleOut','RAnkleIn'
        ]
    )
    # Adds the new keypoint data to the dataframe
    data_3D['LElbow'] = left_elbow_3D
    data_3D['RElbow'] = right_elbow_3D
    data_3D['LWrist'] = left_wrist_3D
    data_3D['RWrist'] = right_wrist_3D
    data_3D['LKnee'] = left_knee_3D
    data_3D['RKnee'] = right_knee_3D
    data_3D['LAnkle'] = left_ankle_3D
    data_3D['RAnkle'] = right_ankle_3D

    # Remove every other frame, our videodata Miqus is 85hz and the data is 170hz
    # OBS! (3 rows corresponds to 1 frame.)
    remove = True
    for i in range(0, data_3D.shape[0]):
        if i % 3 == 0:
            remove = not(remove)

        if remove:
            data_3D = data_3D.drop(i)
    
    data_3D = data_3D.reset_index(drop=True)

    if args.cut_frame is not None:
        # Cuts the data by the same frame as we cut the video. 
        data_3D = data_3D[:args.cut_frame]

    # Convert 3D world to 2D camera coordinates
    data_2D = convert_to_2D(data_3D, args.camera)

    # Creates output names that depends on the name of the data file 
    data_file_name = os.path.basename(
        os.path.normpath(args.data_file)).rsplit(".")[0]
    out_2D = os.path.join(
        args.output_dir, data_file_name + '_2D_keypoints.csv')
    out_3D = os.path.join(
        args.output_dir, data_file_name + '_3D_keypoints.csv')

    # Save the keypoint data as csv files
    # TODO: Add reformat to 2D data i.e 3DWorld -> 3DCamera -> 2D (projection)
    # pd.DataFrame.to_csv(data_2D, path_or_buf=out_2D)
    # TODO: Check if it is better to save this as npz instead. 
    pd.DataFrame.to_csv(data_3D, path_or_buf=out_3D)

In [ ]:
#export
try: from nbdev.imports import IN_NOTEBOOK
except: IN_NOTEBOOK=False

if __name__ == '__main__' and not IN_NOTEBOOK:
    args = parse_args()
    main(args)

In [11]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_model.ipynb.
Converted 01_loss.ipynb.
Converted 02_skeleton.ipynb.
Converted 03_mocap_dataset.ipynb.
Converted 04_h36m_dataset.ipynb.
Converted 05_camera.ipynb.
Converted 06_quaternion.ipynb.
Converted 07_utils.ipynb.
Converted 08_generators.ipynb.
Converted 09_custom_dataset.ipynb.
Converted 10_visualization.ipynb.
Converted 11_arguments.ipynb.
Converted 12_data_utils.ipynb.
Converted 13_prepare_data_2d_custom.ipynb.
Converted 14_infer_video.ipynb.
Converted 15_prepare_data_COCO.ipynb.
Converted 16_pycococreatortools.ipynb.
Converted 17_format_qtmdata.ipynb.
Converted 18_runningpose_dataset.ipynb.
Converted 19_train_detectron2.ipynb.
Converted index.ipynb.
